In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import PIL
import pickle
from PIL import *
import cv2
import tensorflow as tf
import keras
from keras.applications import DenseNet121 # 2017 architecture
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from keras import *
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, optimizers
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
facialexpression_df = pd.read_csv('/content/drive/MyDrive/DL Facial Recognition /icml_face_data.csv')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
facialexpression_df = facialexpression_df.drop(columns = " Usage")


In [ ]:
facialexpression_df.head()

In [ ]:
def string2array(x):
    return np.array(x.split(' ')).reshape(48,48,1).astype('float32')

In [ ]:
def resize(x):
    img = x.reshape(48,48)
    temp = cv2.resize(img, dsize=(48,48), interpolation = cv2.INTER_CUBIC)
    return np.stack([temp,temp,temp], axis = 2)

In [ ]:
facialexpression_df[' pixels'] = facialexpression_df[' pixels'].apply(lambda x : string2array(x))
facialexpression_df[' pixels'] =  facialexpression_df[' pixels'].apply(lambda x : resize(x))

In [ ]:
facialexpression_df.shape

In [ ]:
label_to_text = {0: 'anger', 1 : 'disgust', 2 : 'fear', 3 : 'happiness', 4: 'sad', 5: 'surprise', 6: 'neutral'}

In [ ]:
facialexpression_df[' pixels'][1].shape

In [ ]:
emotions = [1]
for i in emotions:
    data = facialexpression_df[facialexpression_df['emotion']==i][:1]

    img = data[' pixels'].item()
    img = img.reshape(48,48, 3)

    plt.figure()
    plt.title(label_to_text[i])
    plt.imshow(img/255)
    plt.axis('off')

In [ ]:
facialexpression_df.emotion.value_counts().index

In [ ]:
facialexpression_df.emotion.value_counts()

In [ ]:
class1 = facialexpression_df.loc[facialexpression_df['emotion'] == 1]
class1.shape

In [ ]:
from keras.utils import to_categorical

X = facialexpression_df[' pixels']
y = to_categorical(facialexpression_df['emotion'])

X = np.stack(X, axis = 0)
X = X.reshape(35887,48,48,3)

print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, shuffle = True)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train = X_train/255
X_val   = X_val /255
X_test  = X_test/255

In [ ]:
train_datagen = ImageDataGenerator(
rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = "nearest")

In [ ]:
from keras.applications import VGG16

input_shape = (48,48, 3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

VGGmodel = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(7, activation='sigmoid')
])

VGGmodel.summary()

In [ ]:
VGGmodel.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
# using early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath = "/content/drive/MyDrive/DL Facial Recognition /VGGFacialExpression_weights.hdf5", verbose = 1, save_best_only=True)

In [ ]:
history = VGGmodel.fit(train_datagen.flow(X_train, y_train, batch_size=64),
	validation_data= (X_val, y_val), steps_per_epoch=len(X_train) // 64,
	epochs= 100, callbacks=[checkpointer, earlystopping])

In [ ]:
score = VGGmodel.evaluate(X_test, y_test)
print('Test Accuracy: {}'.format(score[1]))

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

In [ ]:
predicted_classes = np.argmax(VGGmodel.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, predicted_classes)
plt.figure(figsize = (10, 10))
sns.heatmap(cm, annot = True, cbar = False)

In [ ]:
L = 5
W = 5

fig, axes = plt.subplots(L, W, figsize = (24, 24))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_test[i], cmap = 'gray')
    axes[i].set_title('Prediction = {}\n True = {}'.format(label_to_text[predicted_classes[i]], label_to_text[y_true[i]]))
    axes[i].axis('off')

plt.subplots_adjust(wspace = 1)

In [ ]:
# Compute accuracy
accuracy = np.mean(predicted_classes == y_true)

print("Accuracy:", accuracy)